# Домашнее задание № 7

In [1]:
#import adagram
from lxml import html
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from pymorphy2 import MorphAnalyzer
from string import punctuation
import json, os, re, sys
from collections import Counter
import numpy as np
from razdel import tokenize as razdel_tokenize
from matplotlib import pyplot as plt
import gensim
import pandas as pd
from sklearn.metrics import adjusted_rand_score
import tqdm
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold


%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [token.text.strip(punct) for token in list(razdel_tokenize(text))]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return words


### Задание 1 Реализовать алгоритм Леска и проверить его на реальном датасете (8 баллов)

Ворднет можно использовать для дизамбигуации. Самый простой алгоритм дизамбигуации - алгоритм Леска. В нём нужное значение слова находится через пересечение слов контекста, в котором употреблено это слово, с определениями значений слова из ворднета. Значение с максимальным пересечением - нужное.

Реализуйте его

In [2]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Светлана\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
def lesk( word, sentence ):
    """Ваш код тут"""
    bestsense = 0
    maxoverlap = 0
    contexts = normalize(sentence)
    
    synsets = wn.synsets(word)
    for i, syns in enumerate(synsets):
        definition = syns.definition()
        definition = normalize(definition)
        overlap = len(list(set(contexts) & set(definition)))
        if overlap > maxoverlap:
            bestsense = i
            maxoverlap = overlap        
    return bestsense


Работать функция должна как-то так:

In [4]:
# на вход подается элемент результата работы уже написанной вами функции get_words_in_context
lesk('day', 'some point or period in time') # для примера контекст совпадает с одним из определений
# а на выходе индекс подходящего синсета

1

In [5]:
# с помощью этого индекса достаем нужный синсет
wn.synsets('day')[1].definition()

'some point or period in time'

**Проверьте насколько хорошо работает такой метод на датасете из семинара**

In [6]:
corpus_wsd = []
corpus = open('corpus_wsd_50k.txt').read().split('\n\n')
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])

Корпус состоит из предложений, где у каждого слова три поля - значение, лемма и само слово. Значение пустое, когда слово однозначное, а у многозначных слов стоит тэг вида **'long%3:00:02::'** Это тэг wordnet'ного формата

In [7]:
true = 0
total = 0
for sent in corpus_wsd[:1000]:
    context = ' '.join([w[-1] for w in sent])
    #print(context)
    for word in sent:
        if word[0] != '':
            total+=1
            pred = lesk( word[1], context )
            if wn.synsets(word[1])[pred] == wn.lemma_from_key(word[0]).synset():
                true+=1
accuracy = true/total
accuracy            
            

0.367619926199262

Когда с помощью алгоритма Леска вы найдете подходящее значение, их можно просто сравнить

**Вам нужно для каждого многозначного слова (т.е. у него есть тэг в первом поле) с помощью алгоритма Леска предсказать нужный синсет и сравнить с правильным. Посчитайте процент правильных предсказаний (accuracy).**

Если считается слишком долго, возьмите поменьше предложений (например, только тысячу)

### Задание 2* (2 балла)

В семинаре для WSI на данных Диалога использовался только Fastext. Попробуйте заменить его на адаграм (обучите свою модель или используйте предобученную out.pkl или https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib), а также поэкспериментируйте с разными алгоритмами кластеризации и их параметрами (можно использовать только те алгоритмы, которые обучаются достаточно быстро)

Для каждого эксперимента рассчитайте ARI

In [8]:
import adagram
import pandas as pd
from sklearn.cluster import *
import wget



In [9]:
vm = adagram.VectorModel.load("out.pkl")

In [10]:
def get_embedding_adagram(text, model, window=5, dim=100):
    
    
    word2context = []
    for i in range(len(text)-1):
        left = max(0, i-window)
        word = text[i]
        left_context = text[left:i]
        right_context = text[i+1:i+window]
        context = left_context + right_context
        word2context.append((word, context))
    
    
    
    vectors = np.zeros((len(word2context), dim))
    
    for i,word in enumerate(word2context):
        word, context = word
        try:
            sense = model.disambiguate(word, context).argmax()
            v = model.sense_vector(word, sense)
            vectors[i] = v 

        except (KeyError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector
        

In [11]:
url = 'https://raw.githubusercontent.com/nlpub/russe-wsi-kit/master/data/main/wiki-wiki/train.csv'
wget.download(url, 'train.csv')

100% [............................................................................] 365614 / 365614

'train.csv'

In [12]:
df = pd.read_csv('train.csv', sep='\t', error_bad_lines=False)


In [13]:
grouped_df = df.groupby('word')[['word', 'context', 'gold_sense_id']]

In [14]:
def count_ari(cluster):
    ARI = []

    for key, _ in grouped_df:
        texts = grouped_df.get_group(key)['context'].apply(normalize)
        X = np.zeros((len(texts), 100))

        for i, text in enumerate(texts):
            text = [word for word in text if word != key]
            X[i] = get_embedding_adagram(text, vm)

        
        cluster.fit(X)
        labels = np.array(cluster.labels_)+1

        ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))

    return(np.mean(ARI))

In [15]:
count_ari(AffinityPropagation(damping=0.9))

0.04266132797411952

In [16]:
count_ari(KMeans(3))

0.21416506362837714

In [17]:
count_ari(DBSCAN(min_samples=1, eps=0.1))

0.0011175070915487569

лучший результат у Kmeans